In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


## Growth Mart DDL
- The Table definitions.
- Creation of schema

In [ ]:
CREATE SCHEMA IF NOT EXISTS growth_mart;

-- ============================================
-- DIMENSION TABLES
-- ============================================

-- Dimension: dim_date
-- Standard date dimension for time-based analysis
CREATE OR REPLACE TABLE growth_mart.dim_date (
    date_key            INTEGER PRIMARY KEY,      -- YYYYMMDD format
    date_actual         DATE NOT NULL,
    day_of_week         INTEGER,
    day_name            VARCHAR(10),
    day_of_month        INTEGER,
    day_of_year         INTEGER,
    week_of_year        INTEGER,
    month_actual        INTEGER,
    month_name          VARCHAR(10),
    quarter_actual      INTEGER,
    quarter_name        VARCHAR(10),
    year_actual         INTEGER,
    is_weekend          BOOLEAN
);

-- Dimension: dim_channel
-- Marketing channels with UTM source mapping
CREATE OR REPLACE TABLE growth_mart.dim_channel (
    channel_key         INTEGER PRIMARY KEY AUTOINCREMENT,
    channel_name        VARCHAR(50) NOT NULL,     -- e.g., 'Google Ads', 'LinkedIn', 'Meta'
    channel_category    VARCHAR(50),              -- e.g., 'Paid Search', 'Paid Social', 'Organic', 'Direct'
    utm_source          VARCHAR(50),              -- e.g., 'google', 'linkedin', 'facebook'
    is_paid             BOOLEAN
);

-- Dimension: dim_campaign
-- Campaign-level attributes from ad platforms
CREATE OR REPLACE TABLE growth_mart.dim_campaign (
    campaign_key        INTEGER PRIMARY KEY AUTOINCREMENT,
    campaign_id         VARCHAR(100) NOT NULL,    -- Natural key from ad_spend
    utm_campaign        VARCHAR(100),
    channel_key         INTEGER,                  -- FK to dim_channel
    campaign_name       VARCHAR(255),             -- Derived/friendly name
    FOREIGN KEY (channel_key) REFERENCES growth_mart.dim_channel(channel_key)
);

-- Dimension: dim_landing_page
-- Landing pages from web analytics
CREATE OR REPLACE TABLE growth_mart.dim_landing_page (
    landing_page_key    INTEGER PRIMARY KEY AUTOINCREMENT,
    landing_page_path   VARCHAR(255) NOT NULL,    -- e.g., '/pricing', '/demo', '/blog'
    page_category       VARCHAR(50),              -- e.g., 'Product', 'Content', 'Conversion'
    is_conversion_page  BOOLEAN
);

-- Dimension: dim_utm
-- UTM parameter combinations for attribution
CREATE OR REPLACE TABLE growth_mart.dim_utm (
    utm_key             INTEGER PRIMARY KEY AUTOINCREMENT,
    utm_source          VARCHAR(50),
    utm_campaign        VARCHAR(100),
    channel_key         INTEGER,                  -- FK to dim_channel
    FOREIGN KEY (channel_key) REFERENCES growth_mart.dim_channel(channel_key)
);

-- ============================================
-- FACT TABLES
-- ============================================

-- Fact: fct_ad_performance
-- Grain: One row per date + campaign_id (daily campaign performance)
CREATE OR REPLACE TABLE growth_mart.fct_ad_performance (
    ad_performance_key  INTEGER PRIMARY KEY AUTOINCREMENT,

    -- Dimension Foreign Keys
    date_key            INTEGER NOT NULL,         -- FK to dim_date
    campaign_key        INTEGER NOT NULL,         -- FK to dim_campaign
    channel_key         INTEGER NOT NULL,         -- FK to dim_channel

    -- Measures
    spend_usd           DECIMAL(12, 2),
    clicks              INTEGER,
    impressions         INTEGER,

    -- Derived Metrics (can also be calculated in views)
    ctr                 DECIMAL(10, 6),           -- Click-through rate (clicks/impressions)
    cpc                 DECIMAL(10, 4),           -- Cost per click (spend/clicks)
    cpm                 DECIMAL(10, 4),           -- Cost per mille (spend/impressions * 1000)

    -- Audit
    loaded_at           TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP(),

    FOREIGN KEY (date_key) REFERENCES growth_mart.dim_date(date_key),
    FOREIGN KEY (campaign_key) REFERENCES growth_mart.dim_campaign(campaign_key),
    FOREIGN KEY (channel_key) REFERENCES growth_mart.dim_channel(channel_key)
);

-- Fact: fct_web_sessions
-- Grain: One row per session_id (session-level web analytics)
CREATE OR REPLACE TABLE growth_mart.fct_web_sessions (
    web_session_key     INTEGER PRIMARY KEY AUTOINCREMENT,
    session_id          VARCHAR(50) NOT NULL,     -- Natural key from web_analytics
    user_id             VARCHAR(50),

    -- Dimension Foreign Keys
    session_date_key    INTEGER NOT NULL,         -- FK to dim_date
    channel_key         INTEGER,                  -- FK to dim_channel (derived from utm_source)
    landing_page_key    INTEGER,                  -- FK to dim_landing_page
    utm_key             INTEGER,                  -- FK to dim_utm

    -- Measures
    pageviews           INTEGER,
    conversions         INTEGER,

    -- Flags
    is_converted        BOOLEAN,                  -- TRUE if conversions > 0
    is_attributed       BOOLEAN,                  -- TRUE if UTM params present

    -- Audit
    loaded_at           TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP(),

    FOREIGN KEY (session_date_key) REFERENCES growth_mart.dim_date(date_key),
    FOREIGN KEY (channel_key) REFERENCES growth_mart.dim_channel(channel_key),
    FOREIGN KEY (landing_page_key) REFERENCES growth_mart.dim_landing_page(landing_page_key),
    FOREIGN KEY (utm_key) REFERENCES growth_mart.dim_utm(utm_key)
);

-- Fact: fct_channel_roi
-- Grain: One row per channel (aggregated totals for ROI calculation)
-- Joins marketing data with closed deals for full-funnel ROI
CREATE OR REPLACE TABLE growth_mart.fct_channel_roi (
    channel_roi_key         INTEGER PRIMARY KEY AUTOINCREMENT,

    -- Dimension Foreign Keys
    channel_key             INTEGER NOT NULL,         -- FK to dim_channel

    -- Period (for filtering)
    period_start_date       DATE,
    period_end_date         DATE,

    -- Marketing Metrics (totals)
    total_spend_usd         DECIMAL(14, 2),
    total_impressions       INTEGER,
    total_clicks            INTEGER,
    total_sessions          INTEGER,
    total_conversions       INTEGER,

    -- Sales Metrics (from salesforce)
    total_opportunities     INTEGER,
    total_closed_won        INTEGER,
    total_closed_lost       INTEGER,
    total_pipeline_value    DECIMAL(14, 2),
    total_closed_won_revenue DECIMAL(14, 2),


    -- Audit
    loaded_at               TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP(),

    FOREIGN KEY (channel_key) REFERENCES growth_mart.dim_channel(channel_key)
);

-- Fact: fct_marketing_funnel
-- Grain: One row per date + channel (daily channel-level funnel metrics)
-- This is an aggregated fact table for funnel/ROI analysis
CREATE OR REPLACE TABLE growth_mart.fct_marketing_funnel (
    funnel_key          INTEGER PRIMARY KEY AUTOINCREMENT,

    -- Dimension Foreign Keys
    date_key            INTEGER NOT NULL,         -- FK to dim_date
    channel_key         INTEGER NOT NULL,         -- FK to dim_channel

    -- Top of Funnel (from ad_spend)
    spend_usd           DECIMAL(12, 2),
    impressions         INTEGER,
    clicks              INTEGER,

    -- Middle of Funnel (from web_analytics)
    sessions            INTEGER,
    pageviews           INTEGER,
    conversions         INTEGER,
    unique_users        INTEGER,

    -- Bottom of Funnel (from salesforce_opportunities)
    opportunities       INTEGER,
    pipeline_value_usd  DECIMAL(14, 2),
    closed_won_opps     INTEGER,
    closed_won_revenue  DECIMAL(14, 2),
    closed_lost_opps    INTEGER,
    -- Audit
    loaded_at           TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP(),

    FOREIGN KEY (date_key) REFERENCES growth_mart.dim_date(date_key),
    FOREIGN KEY (channel_key) REFERENCES growth_mart.dim_channel(channel_key)
);


In [ ]:
-- ============================================
-- VIEWS FOR COMMON ANALYSES
-- ============================================

-- View: Channel Performance Summary
CREATE OR REPLACE TABLE DATA_ENGINEERING_KATA.GROWTH_MART.CHANNEL_PERFORMANCE AS
SELECT
    c.channel_name,
    c.channel_category,
    SUM(f.spend_usd) AS total_spend,
    SUM(f.impressions) AS total_impressions,
    SUM(f.clicks) AS total_clicks,
    SUM(f.sessions) AS total_sessions,
    SUM(f.conversions) AS total_conversions,
    SUM(f.opportunities) AS total_opportunities,
    SUM(f.closed_won_opps) AS total_closed_won,
    SUM(f.closed_won_revenue) AS total_revenue,
    -- Calculated metrics
    CASE WHEN SUM(f.impressions) > 0 THEN SUM(f.clicks) / SUM(f.impressions) END AS overall_ctr,
    CASE WHEN SUM(f.clicks) > 0 THEN SUM(f.spend_usd) / SUM(f.clicks) END AS overall_cpc,
    CASE WHEN SUM(f.closed_won_opps) > 0 THEN SUM(f.spend_usd) / SUM(f.closed_won_opps) END AS overall_cac,
    CASE WHEN SUM(f.spend_usd) > 0 THEN SUM(f.closed_won_revenue) / SUM(f.spend_usd) END AS overall_roas,
    CASE WHEN SUM(f.spend_usd) > 0 THEN (SUM(f.closed_won_revenue) - SUM(f.spend_usd)) / SUM(f.spend_usd) END AS overall_roi
FROM growth_mart.fct_marketing_funnel f
JOIN growth_mart.dim_channel c ON f.channel_key = c.channel_key
GROUP BY c.channel_name, c.channel_category;


In [ ]:

-- View: Weekly Funnel Trends
CREATE OR REPLACE VIEW DATA_ENGINEERING_KATA.GROWTH_MART.v_weekly_funnel_trends AS
SELECT
    d.year_actual,
    d.week_of_year,
    MIN(d.date_actual) AS week_start_date,
    c.channel_name,
    SUM(f.spend_usd) AS weekly_spend,
    SUM(f.clicks) AS weekly_clicks,
    SUM(f.sessions) AS weekly_sessions,
    SUM(f.conversions) AS weekly_conversions,
    SUM(f.closed_won_opps) AS weekly_closed_won,
    SUM(f.closed_won_revenue) AS weekly_revenue
FROM growth_mart.fct_marketing_funnel f
JOIN growth_mart.dim_date d ON f.date_key = d.date_key
JOIN growth_mart.dim_channel c ON f.channel_key = c.channel_key
GROUP BY d.year_actual, d.week_of_year, c.channel_name;


## GROWTH MART -- Load Data

### dim_channel

In [ ]:
-- ============================================
-- Populate dim_channel (dynamic from source data)
-- Derives distinct channels from ad_spend, web_analytics, and salesforce
-- ============================================
INSERT INTO DATA_ENGINEERING_KATA.growth_mart.dim_channel (channel_name, channel_category, utm_source, is_paid)
WITH all_channels AS (
    -- From ad_spend (paid channels with utm_source mapping)
    SELECT DISTINCT
        channel AS channel_name,
        utm_source,
        TRUE AS is_paid
    FROM DATA_ENGINEERING_KATA.RAW_DATA.ad_spend

    UNION

    -- From web_analytics (map utm_source to channel)
    SELECT DISTINCT
        CASE
            WHEN utm_source = 'google' THEN 'Google Ads'
            WHEN utm_source = 'facebook' THEN 'Meta'
            WHEN utm_source = 'linkedin' THEN 'LinkedIn'
            WHEN utm_source = 'twitter' THEN 'Twitter'
            WHEN utm_source IS NULL THEN 'Direct'
            ELSE 'Other'
        END AS channel_name,
        utm_source,
        CASE WHEN utm_source IS NOT NULL THEN TRUE ELSE FALSE END AS is_paid
    FROM DATA_ENGINEERING_KATA.RAW_DATA.web_analytics
    WHERE utm_source NOT IN (SELECT DISTINCT utm_source FROM ad_spend WHERE utm_source IS NOT NULL)
       OR utm_source IS NULL

    UNION

    -- From salesforce opportunities (non-paid sources)
    SELECT DISTINCT
        cust_source AS channel_name,
        NULL AS utm_source,
        FALSE AS is_paid
    FROM DATA_ENGINEERING_KATA.RAW_DATA.salesforce_opportunities
    WHERE cust_source NOT IN (SELECT DISTINCT channel FROM ad_spend)
)
SELECT DISTINCT
    channel_name,
    CASE
        WHEN channel_name = 'Google Ads' THEN 'Paid Search'
        WHEN channel_name IN ('LinkedIn', 'Meta', 'Twitter') THEN 'Paid Social'
        WHEN channel_name = 'Organic' THEN 'Organic'
        WHEN channel_name = 'Direct' THEN 'Direct'
        ELSE 'Other'
    END AS channel_category,
    utm_source,
    is_paid
FROM all_channels;



### dim_campaign

In [ ]:
-- ============================================
-- Populate dim_campaign (from ad_spend)
-- ============================================
INSERT INTO growth_mart.dim_campaign (campaign_id, utm_campaign, channel_key, campaign_name)
SELECT DISTINCT
    a.campaign_id,
    a.utm_campaign,
    c.channel_key,
    REPLACE(REPLACE(a.campaign_id, 'CAMP_', ''), '_', ' ') AS campaign_name
FROM DATA_ENGINEERING_KATA.RAW_DATA.ad_spend a
LEFT JOIN growth_mart.dim_channel c ON a.channel = c.channel_name;



### dim_landing_page

In [ ]:
INSERT INTO growth_mart.dim_landing_page (landing_page_path, page_category)
SELECT DISTINCT
    landing_page AS landing_page_path,
    CASE
        WHEN landing_page = '/pricing' THEN 'Pricing'
        WHEN landing_page = '/demo' THEN 'Demo'
        WHEN landing_page = '/enterprise' THEN 'Enterprise'
        WHEN landing_page = '/solutions' THEN 'Solutions'
        WHEN landing_page = '/blog' THEN 'Blog'
        WHEN landing_page = '/' THEN 'Homepage'
        ELSE 'Other'
    END AS page_category
FROM web_analytics;

### dim_utm

In [ ]:
INSERT INTO growth_mart.dim_utm (utm_source, utm_campaign, channel_key)
SELECT DISTINCT
    COALESCE(a.utm_source, w.utm_source) AS utm_source,
    COALESCE(a.utm_campaign, w.utm_campaign) AS utm_campaign,
    c.channel_key
FROM ad_spend a
FULL OUTER JOIN (
    SELECT DISTINCT utm_source, utm_campaign
    FROM web_analytics
    WHERE utm_source IS NOT NULL
) w ON a.utm_source = w.utm_source AND a.utm_campaign = w.utm_campaign
LEFT JOIN growth_mart.dim_channel c ON COALESCE(a.utm_source, w.utm_source) = c.utm_source;

### FACT TABLE - ad_performance

In [ ]:
INSERT INTO growth_mart.fct_ad_performance (
    date_key,
    campaign_key,
    channel_key,
    spend_usd,
    clicks,
    impressions,
    ctr,
    cpc,
    cpm
)
SELECT
    TO_NUMBER(TO_CHAR(a.date, 'YYYYMMDD')) AS date_key,
    cp.campaign_key,
    c.channel_key,
    a.spend_usd,
    a.clicks,
    a.impressions,
    -- CTR: Click-through rate (clicks / impressions)
    CASE
        WHEN a.impressions > 0 THEN ROUND((a.clicks::DECIMAL / a.impressions) * 100, 4)
        ELSE 0
    END AS ctr,
    -- CPC: Cost per click (spend / clicks)
    CASE
        WHEN a.clicks > 0 THEN ROUND(a.spend_usd / a.clicks, 2)
        ELSE 0
    END AS cpc,
    -- CPM: Cost per thousand impressions (spend / impressions * 1000)
    CASE
        WHEN a.impressions > 0 THEN ROUND((a.spend_usd / a.impressions) * 1000, 2)
        ELSE 0
    END AS cpm
FROM ad_spend a
LEFT JOIN growth_mart.dim_channel c ON a.channel = c.channel_name
LEFT JOIN growth_mart.dim_campaign cp ON a.campaign_id = cp.campaign_id;

### FACT TABLE - web_sessions

In [ ]:
INSERT INTO growth_mart.fct_web_sessions (
    session_id,
    user_id,
    session_date_key,
    channel_key,
    landing_page_key,
    utm_key,
    pageviews,
    conversions,
    is_converted
)
SELECT
    w.session_id,
    w.user_id,
    TO_NUMBER(TO_CHAR(w.session_date, 'YYYYMMDD')) AS session_date_key,
    c.channel_key,
    lp.landing_page_key,
    u.utm_key,
    w.pageviews,
    w.conversions,
    CASE WHEN w.conversions > 0 THEN TRUE ELSE FALSE END AS is_converted
FROM web_analytics w
LEFT JOIN growth_mart.dim_channel c ON w.utm_source = c.utm_source
LEFT JOIN growth_mart.dim_landing_page lp ON w.landing_page = lp.landing_page_path
LEFT JOIN growth_mart.dim_utm u ON w.utm_source = u.utm_source AND w.utm_campaign = u.utm_campaign;

### FACT TABLE -- marketing_funnel

In [ ]:
INSERT INTO growth_mart.fct_marketing_funnel (
    date_key,
    channel_key,
    spend_usd,
    impressions,
    clicks,
    sessions,
    pageviews,
    conversions,
    unique_users,
    opportunities,
    pipeline_value_usd,
    closed_won_opps,
    closed_won_revenue,
    closed_lost_opps,
    loaded_at
)
WITH ad_metrics AS (
    SELECT
        TO_NUMBER(TO_CHAR(date, 'YYYYMMDD')) AS date_key,
        channel,
        SUM(spend_usd) AS total_spend,
        SUM(impressions) AS total_impressions,
        SUM(clicks) AS total_clicks
    FROM raw_data.ad_spend
    GROUP BY TO_NUMBER(TO_CHAR(date, 'YYYYMMDD')), channel
),

web_metrics AS (
    SELECT
        TO_NUMBER(TO_CHAR(session_date, 'YYYYMMDD')) AS date_key,
        CASE
            WHEN utm_source = 'google' THEN 'Google Ads'
            WHEN utm_source = 'facebook' THEN 'Meta'
            WHEN utm_source = 'linkedin' THEN 'LinkedIn'
            WHEN utm_source = 'twitter' THEN 'Twitter'
            WHEN utm_source IS NULL THEN 'Direct'
            ELSE 'Other'
        END AS channel,
        COUNT(DISTINCT session_id) AS total_sessions,
        SUM(pageviews) AS total_pageviews,
        SUM(conversions) AS total_conversions,
        COUNT(DISTINCT user_id) AS total_unique_users
    FROM raw_data.web_analytics
    GROUP BY 
        TO_NUMBER(TO_CHAR(session_date, 'YYYYMMDD')),
        CASE
            WHEN utm_source = 'google' THEN 'Google Ads'
            WHEN utm_source = 'facebook' THEN 'Meta'
            WHEN utm_source = 'linkedin' THEN 'LinkedIn'
            WHEN utm_source = 'twitter' THEN 'Twitter'
            WHEN utm_source IS NULL THEN 'Direct'
            ELSE 'Other'
        END
),

sf_opps AS (
    SELECT
        TO_NUMBER(TO_CHAR(created_date, 'YYYYMMDD')) AS date_key,
        cust_source AS channel,
        COUNT(opportunity_id) AS total_opportunities,
        SUM(amount_usd) AS total_pipeline_value,
        SUM(CASE WHEN cust_stage = 'Closed Won' THEN 1 ELSE 0 END) AS total_closed_won,
        SUM(CASE WHEN cust_stage = 'Closed Won' THEN amount_usd ELSE 0 END) AS total_closed_won_revenue,
        SUM(CASE WHEN cust_stage = 'Closed Lost' THEN 1 ELSE 0 END) AS total_closed_lost
    FROM raw_data.salesforce_opportunities
    GROUP BY 
        TO_NUMBER(TO_CHAR(created_date, 'YYYYMMDD')),
        cust_source
),

-- Combine all date/channel combinations
all_keys AS (
    SELECT date_key, channel FROM ad_metrics
    UNION
    SELECT date_key, channel FROM web_metrics
    UNION
    SELECT date_key, channel FROM sf_opps
)

SELECT
    k.date_key,
    c.channel_key,
    COALESCE(a.total_spend, 0) AS spend_usd,
    COALESCE(a.total_impressions, 0) AS impressions,
    COALESCE(a.total_clicks, 0) AS clicks,
    COALESCE(w.total_sessions, 0) AS sessions,
    COALESCE(w.total_pageviews, 0) AS pageviews,
    COALESCE(w.total_conversions, 0) AS conversions,
    COALESCE(w.total_unique_users, 0) AS unique_users,
    COALESCE(o.total_opportunities, 0) AS opportunities,
    COALESCE(o.total_pipeline_value, 0) AS pipeline_value_usd,
    COALESCE(o.total_closed_won, 0) AS closed_won_opps,
    COALESCE(o.total_closed_won_revenue, 0) AS closed_won_revenue,
    COALESCE(o.total_closed_lost, 0) AS closed_lost_opps,
    CURRENT_TIMESTAMP() AS loaded_at
FROM all_keys k
LEFT JOIN ad_metrics a 
    ON k.date_key = a.date_key AND k.channel = a.channel
LEFT JOIN web_metrics w 
    ON k.date_key = w.date_key AND k.channel = w.channel
LEFT JOIN sf_opps o 
    ON k.date_key = o.date_key AND k.channel = o.channel
LEFT JOIN growth_mart.dim_channel c 
    ON k.channel = c.channel_name
WHERE c.channel_key IS NOT NULL;

# SALES MART

In [ ]:
CREATE OR REPLACE TABLE DATA_ENGINEERING_KATA.SALES_MART.DIM_DATE (
    date_key            INTEGER PRIMARY KEY,      -- YYYYMMDD format
    date_actual         DATE NOT NULL,
    day_of_week         INTEGER,
    day_name            VARCHAR(10),
    day_of_month        INTEGER,
    day_of_year         INTEGER,
    week_of_year        INTEGER,
    month_actual        INTEGER,
    month_name          VARCHAR(10),
    quarter_actual      INTEGER,
    quarter_name        VARCHAR(10),
    year_actual         INTEGER,
    is_weekend          BOOLEAN
);

In [ ]:
USE DATABASE DATA_ENGINEERING_KATA;

CREATE OR REPLACE TABLE sales_mart.stg_ad_spend (
    date            DATE,
    campaign_id     VARCHAR(100),
    channel         VARCHAR(50),
    utm_source      VARCHAR(50),
    utm_campaign    VARCHAR(100),
    spend_usd       DECIMAL(12, 2),
    clicks          INTEGER,
    impressions     INTEGER
);

-- Table: salesforce_opportunities (raw)
CREATE OR REPLACE TABLE sales_mart.stg_salesforce_opportunities (
    opportunity_id  VARCHAR(50),
    account_id      VARCHAR(50),
    created_date    DATE,
    stage           VARCHAR(50),
    amount_usd      DECIMAL(12, 2),
    source          VARCHAR(50),
    owner_region    VARCHAR(50)
);

CREATE OR REPLACE TABLE sales_mart.stg_web_analytics (
    session_id      VARCHAR(50),
    user_id         VARCHAR(50),
    session_date    DATE,
    landing_page    VARCHAR(255),
    utm_source      VARCHAR(50),
    utm_campaign    VARCHAR(100),
    pageviews       INTEGER,
    conversions     INTEGER
);

-- ============================================
-- DIMENSION TABLES
-- ============================================

-- Dimension: dim_date
CREATE OR REPLACE TABLE sales_mart.dim_date (
    date_key            INTEGER PRIMARY KEY,      -- YYYYMMDD format
    date_actual         DATE NOT NULL,
    day_of_week         INTEGER,
    day_name            VARCHAR(10),
    day_of_month        INTEGER,
    day_of_year         INTEGER,
    week_of_year        INTEGER,
    month_actual        INTEGER,
    month_name          VARCHAR(10),
    quarter_actual      INTEGER,
    quarter_name        VARCHAR(10),
    year_actual         INTEGER,
    is_weekend          BOOLEAN
);

-- Dimension: dim_channel
CREATE OR REPLACE TABLE sales_mart.dim_channel (
    channel_key         INTEGER PRIMARY KEY AUTOINCREMENT,
    channel_name        VARCHAR(50) NOT NULL,     -- e.g., 'Google Ads', 'LinkedIn', 'Meta'
    channel_category    VARCHAR(50),              -- e.g., 'Paid Search', 'Paid Social', 'Organic', 'Direct'
    utm_source          VARCHAR(50),              -- e.g., 'google', 'linkedin', 'facebook'
    is_paid             BOOLEAN
);

-- Dimension: dim_campaign
CREATE OR REPLACE TABLE sales_mart.dim_campaign (
    campaign_key        INTEGER PRIMARY KEY AUTOINCREMENT,
    campaign_id         VARCHAR(100) NOT NULL,    -- Natural key from ad_spend
    utm_campaign        VARCHAR(100),
    channel_key         INTEGER,                  -- FK to dim_channel
    campaign_name       VARCHAR(255),             -- Derived/friendly name
    FOREIGN KEY (channel_key) REFERENCES sales_mart.dim_channel(channel_key)
);

-- Dimension: dim_deal_stage
CREATE OR REPLACE TABLE sales_mart.dim_deal_stage (
    stage_key           INTEGER PRIMARY KEY AUTOINCREMENT,
    stage_name          VARCHAR(50) NOT NULL,     -- 'Pipeline', 'Proposal', 'Closed Won', 'Closed Lost'
    stage_category      VARCHAR(50),              -- 'Open', 'Closed'
    is_closed           BOOLEAN,
    is_won              BOOLEAN,
    stage_order         INTEGER                   -- For sorting in reports
);

-- Dimension: dim_sales_region
-- Note: Source data only has region, not individual sales rep details
CREATE OR REPLACE TABLE sales_mart.dim_sales_region (
    region_key          INTEGER PRIMARY KEY AUTOINCREMENT,
    region_name         VARCHAR(50) NOT NULL,     -- 'North America', 'Europe', 'APAC'
    region_code         VARCHAR(10)
);

-- ============================================
-- FACT TABLES
-- ============================================

-- Fact: fct_opportunities
-- Grain: One row per opportunity
CREATE OR REPLACE TABLE sales_mart.fct_opportunities (
    opportunity_key     INTEGER PRIMARY KEY AUTOINCREMENT,
    opportunity_id      VARCHAR(50) NOT NULL,     -- Natural key from Salesforce
    account_id          VARCHAR(50),

    -- Dimension Foreign Keys
    created_date_key    INTEGER NOT NULL,         -- FK to dim_date
    channel_key         INTEGER,                  -- FK to dim_channel (derived from source)
    stage_key           INTEGER NOT NULL,         -- FK to dim_deal_stage
    region_key          INTEGER,                  -- FK to dim_sales_region

    -- Measures
    amount_usd          DECIMAL(12, 2),

    -- Audit
    loaded_at           TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP(),

    FOREIGN KEY (created_date_key) REFERENCES sales_mart.dim_date(date_key),
    FOREIGN KEY (channel_key) REFERENCES sales_mart.dim_channel(channel_key),
    FOREIGN KEY (stage_key) REFERENCES sales_mart.dim_deal_stage(stage_key),
    FOREIGN KEY (region_key) REFERENCES sales_mart.dim_sales_region(region_key)
);

-- Fact: fct_closed_deals
-- Grain: One row per closed-won opportunity
CREATE OR REPLACE TABLE sales_mart.fct_closed_deals (
    closed_deal_key     INTEGER PRIMARY KEY AUTOINCREMENT,
    opportunity_id      VARCHAR(50) NOT NULL,     -- Natural key from Salesforce
    account_id          VARCHAR(50),

    -- Dimension Foreign Keys
    closed_date_key     INTEGER NOT NULL,         -- FK to dim_date
    channel_key         INTEGER,                  -- FK to dim_channel
    region_key          INTEGER,                  -- FK to dim_sales_region

    -- Measures
    deal_amount_usd     DECIMAL(12, 2),

    -- Audit
    loaded_at           TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP(),

    FOREIGN KEY (closed_date_key) REFERENCES sales_mart.dim_date(date_key),
    FOREIGN KEY (channel_key) REFERENCES sales_mart.dim_channel(channel_key),
    FOREIGN KEY (region_key) REFERENCES sales_mart.dim_sales_region(region_key)
);


## POPULATE SALES MART TABLES

### dim_date

In [ ]:
-- ============================================
-- SALES MART - DATA POPULATION SCRIPTS
-- ============================================
-- This file contains all INSERT/transformation logic
-- to populate the Sales Mart dimensional model
-- ============================================

-- ============================================
-- Populate dim_date (for date range in your data)
-- ============================================
INSERT INTO sales_mart.dim_date
SELECT
    TO_NUMBER(TO_CHAR(date_actual, 'YYYYMMDD')) AS date_key,
    date_actual,
    DAYOFWEEK(date_actual) AS day_of_week,
    DAYNAME(date_actual) AS day_name,
    DAY(date_actual) AS day_of_month,
    DAYOFYEAR(date_actual) AS day_of_year,
    WEEKOFYEAR(date_actual) AS week_of_year,
    MONTH(date_actual) AS month_actual,
    MONTHNAME(date_actual) AS month_name,
    QUARTER(date_actual) AS quarter_actual,
    'Q' || QUARTER(date_actual) AS quarter_name,
    YEAR(date_actual) AS year_actual,
    CASE WHEN DAYOFWEEK(date_actual) IN (0, 6) THEN TRUE ELSE FALSE END AS is_weekend
FROM (
    SELECT DATEADD(DAY, SEQ4(), '2025-01-01')::DATE AS date_actual
    FROM TABLE(GENERATOR(ROWCOUNT => 365))
);



### dim_channel

In [ ]:
-- ============================================
-- Populate dim_channel (dynamic from source data)
-- Derives distinct channels from both ad_spend and salesforce_opportunities
-- ============================================
INSERT INTO sales_mart.dim_channel (channel_name, channel_category, utm_source, is_paid)
WITH all_channels AS (
    -- From ad_spend (paid channels with utm_source mapping)
    SELECT DISTINCT
        channel AS channel_name,
        utm_source,
        TRUE AS is_paid
    FROM DATA_ENGINEERING_KATA.RAW_DATA.AD_SPEND

    UNION

    -- From salesforce opportunities (includes non-paid sources)
    SELECT DISTINCT
        cust_source AS channel_name,
        NULL AS utm_source,
        FALSE AS is_paid
    FROM DATA_ENGINEERING_KATA.RAW_DATA.SALESFORCE_OPPORTUNITIES
    WHERE cust_source NOT IN (SELECT DISTINCT channel FROM DATA_ENGINEERING_KATA.RAW_DATA.AD_SPEND)
)
SELECT
    channel_name,
    CASE
        WHEN channel_name = 'Google Ads' THEN 'Paid Search'
        WHEN channel_name IN ('LinkedIn', 'Meta', 'Twitter') THEN 'Paid Social'
        WHEN channel_name = 'Organic' THEN 'Organic'
        WHEN channel_name = 'Direct' THEN 'Direct'
        ELSE 'Other'
    END AS channel_category,
    utm_source,
    is_paid
FROM all_channels;



### dim_deal_stage

In [ ]:
-- ============================================
-- Populate dim_deal_stage
-- ============================================
INSERT INTO sales_mart.dim_deal_stage (stage_name, stage_category, is_closed, is_won, stage_order)
VALUES
    ('Pipeline', 'Open', FALSE, FALSE, 1),
    ('Proposal', 'Open', FALSE, FALSE, 2),
    ('Closed Won', 'Closed', TRUE, TRUE, 3),
    ('Closed Lost', 'Closed', TRUE, FALSE, 4);



### dim_sales_region

In [ ]:
-- ============================================
-- Populate dim_sales_region
-- ============================================
INSERT INTO sales_mart.dim_sales_region (region_name, region_code)
VALUES
    ('North America', 'NA'),
    ('Europe', 'EU'),
    ('APAC', 'APAC');


### dim_campaign

In [ ]:
-- ============================================
-- Populate dim_campaign (from ad_spend)
-- ============================================
INSERT INTO sales_mart.dim_campaign (campaign_id, utm_campaign, channel_key, campaign_name)
SELECT DISTINCT
    a.campaign_id,
    a.utm_campaign,
    c.channel_key,
    REPLACE(REPLACE(a.campaign_id, 'CAMP_', ''), '_', ' ') AS campaign_name
FROM raw_data.ad_spend a
LEFT JOIN sales_mart.dim_channel c ON a.channel = c.channel_name;

### FACT TABLE - OPPORTUNITIES

In [ ]:
-- ============================================
-- Populate fct_opportunities
-- Grain: One row per opportunity (all opportunities)
-- ============================================
INSERT INTO sales_mart.fct_opportunities (
    opportunity_id,
    account_id,
    created_date_key,
    channel_key,
    stage_key,
    region_key,
    amount_usd
)
SELECT
    o.opportunity_id,
    o.account_id,
    TO_NUMBER(TO_CHAR(o.created_date, 'YYYYMMDD')) AS created_date_key,
    c.channel_key,
    s.stage_key,
    r.region_key,
    o.amount_usd
FROM RAW_DATA.salesforce_opportunities o
LEFT JOIN sales_mart.dim_channel c ON o.cust_source = c.channel_name
LEFT JOIN sales_mart.dim_deal_stage s ON o.cust_stage = s.stage_name
LEFT JOIN sales_mart.dim_sales_region r ON o.owner_region = r.region_name;

### FACE TABLE -- CLOSED DEALS

In [ ]:


-- ============================================
-- Populate fct_closed_deals (Closed Won only)
-- Grain: One row per closed-won opportunity
-- ============================================
INSERT INTO sales_mart.fct_closed_deals (
    opportunity_id,
    account_id,
    closed_date_key,
    channel_key,
    region_key,
    deal_amount_usd
)
SELECT
    o.opportunity_id,
    o.account_id,
    TO_NUMBER(TO_CHAR(o.created_date, 'YYYYMMDD')) AS closed_date_key,
    c.channel_key,
    r.region_key,
    o.amount_usd
FROM raw_data.salesforce_opportunities o
LEFT JOIN sales_mart.dim_channel c ON o.cust_source = c.channel_name
LEFT JOIN sales_mart.dim_sales_region r ON o.owner_region = r.region_name
WHERE o.cust_stage = 'Closed Won';


# EXPLORATORY ANALYSIS


In [ ]:
USE DATABASE DATA_ENGINEERING_KATA;
USE SCHEMA RAW_DATA;

with spend as (
    select 
        date,
        lower(campaign_id) as campaign_name,
        upper(channel) as chan,
        lower(trim(utm_source)) as utm_source,
        lower(trim(utm_campaign)) as utm_campaign,
        spend_usd, 
        clicks,
        impressions
    from ad_spend
), web as (
    select 
        session_id,
        REPLACE(user_id, 'U','') as w_user_id,
        session_date as w_date,
        lower(trim(landing_page)) as path,
        lower(trim(utm_source)) as utm_source,
        lower(trim(utm_campaign)) as utm_campaign,
        pageviews as pv,
        conversions as conv
    from web_analytics

), sf_opps as (
    select 
        opportunity_id as opp_id,
        REPLACE(account_id, 'ACCT','') as acc_id,
        created_date,
        cust_stage,
        amount_usd as amount,
        lower(trim(cust_source)) as source,
        owner_region
    from salesforce_opportunities
) 
select 
    row_number() over(partition by session_id order by session_date asc) as session_num,
    w_user_id,
    session_date,
    w.utm_source as web_source,
    w.utm_campaign as web_campaign,
    
    

from web w LEFT JOIN sf_opps sf ON w.w_user_id = sf.acc_id AND w.w_date = sf.created_date
where sf_id is not null















## Building Views 